# 1. 使用FUNCTION_CALL模式

In [6]:
from langchain.schema import prompt_template
import os

from langchain.agents import create_tool_calling_agent, AgentExecutor

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
import dotenv
from langchain_community.tools import TavilySearchResults


# 获取Tavily搜索的实例
dotenv.load_dotenv()

search = TavilySearchResults(max_results=3)

# 获取对应的工具的实例
search_tool = Tool(
    func=search.run,
    name='Search',
    description='用于检索互联网上的信息',
)

# 获取大模型
llm = ChatOpenAI(
    model_name=os.getenv("MODEL"),
    api_key=os.getenv("ARK_API_KEY"),
    base_url=os.getenv("BASE_URL"),
    temperature=0
)

# 提供提示词模版(以ChatPromptTemplate为例)
prompt_template =  ChatPromptTemplate.from_messages([
    ("system","你是一个乐于助人的ai助手,根据用户的提问,必要时调用Search工具,使用互联网检索数据"),
    ("human","{input}"),
    ("system","{agent_scratchpad}")
])

# 获取Agent的实例: create_tool_calling
agent = create_tool_calling_agent(
    llm=llm,
    prompt=prompt_template,
    tools=[search_tool],
)

# 获取AgentExecutor的实例
agent_exector =  AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True
)

# 通过AgentExecutor的实例调用invoke(),得到响应
result = agent_exector.invoke({"input":"查询今天汕尾的天气情况"})

print(result)






> Entering new AgentExecutor chain...

Invoking: `Search` with `汕尾天气预报`


[{'title': '汕尾 - 中国气象局-天气预报-城市预报', 'url': 'https://weather.cma.cn/web/weather/59501', 'content': '|  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 08:00 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 |\n| 天气 |\n| 气温 | 24.2℃ | 26.2℃ | 29.4℃ | 29.8℃ | 28.4℃ | 27.8℃ | 25.4℃ | 22.6℃ |\n| 降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 2.3mm | 2.3mm | 2.3mm |\n| 风速 | 7.9m/s | 7.9m/s | 5.6m/s | 5.1m/s | 5.2m/s | 7.6m/s | 9.4m/s | 9.3m/s |\n| 风向 | 东北风 | 东北风 | 东北风 | 东北风 | 东北风 | 东北风 | 东北风 | 东北风 |\n| 气压 | 1008.5hPa | 1007.1hPa | 1005.8hPa | 1006.5hPa | 1007.3hPa | 1007.4hPa | 1007.5hPa | 1008.2hPa | [...] |  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 08:00 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 |\n| 天气 |\n| 气温 | 20.2℃ | 21.8℃ | 23.7℃ | 23.8℃ | 23.7℃ | 21.8℃ | 21℃ | 20.2℃ |\n| 降水 | 无降水 | 2.3mm | 2.3mm | 2.3mm | 2.3mm | 无降水 | 无降水 | 无降水 |\n| 风速 | 7.5m/s | 7.7m/s | 7.5m/s | 7.9m/s | 7.5m/s

## 2. 使用ReAct模式

## 举例1: 使用PromptTemplate实现

In [2]:
from langchain.schema import prompt_template
import os

from langchain.agents import create_tool_calling_agent, AgentExecutor, create_react_agent

from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
import dotenv
from langchain_community.tools import TavilySearchResults


# 获取Tavily搜索的实例
dotenv.load_dotenv()

search = TavilySearchResults(max_results=3)

# 获取对应的工具的实例
search_tool = Tool(
    func=search.run,
    name='Search',
    description='用于检索互联网上的信息',
)

# 获取大模型
llm = ChatOpenAI(
    model_name=os.getenv("MODEL"),
    api_key=os.getenv("ARK_API_KEY"),
    base_url=os.getenv("BASE_URL"),
    temperature=0
)

# 提供提示词模版(以PromptTemplate为例)
template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

prompt_template =  PromptTemplate.from_template(
    template=template,
)

# 获取Agent的实例: create_tool_calling
agent = create_react_agent(
    llm=llm,
    prompt=prompt_template,
    tools=[search_tool],
)

# 获取AgentExecutor的实例
agent_exector =  AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True
)

# 通过AgentExecutor的实例调用invoke(),得到响应
result = agent_exector.invoke({"input":"查询今天汕尾的天气情况"})

print(result)






> Entering new AgentExecutor chain...
Action: Search  
Action Input: '汕尾天气情况 2023年10月25日'  [{'title': '23到25日平稳回温早晚清凉 - 广东- 中国天气网', 'url': 'http://gd.weather.com.cn/syxw/10/3661033.shtml', 'content': ':   曼谷 东京 首尔 吉隆坡 新加坡 巴黎 罗马 伦敦 雅典 柏林 纽约 温哥华 墨西哥城 哈瓦那 圣何塞 巴西利亚 布宜诺斯艾利斯 圣地亚哥 利马 基多 悉尼 墨尔本 惠灵顿 奥克兰 苏瓦 开罗 内罗毕 开普敦 维多利亚 拉巴特\n\n选择洲际\n\n:   亚洲 欧洲 北美洲 南美洲 非洲 大洋洲\n\n23到25日平稳回温 早晚清凉\n\n### 【字体：大;) 中;) 小;)】 2023-10-23 11:27:22   来源： 中国天气网广东站\n\n预计，23到25日我省大部多云为主，各地气温继续回升，早晚天气清凉，部分市县有（轻）雾。\n\n天气实况：22日我省大部阴天为主，出现了小雨，湛江西北部、茂名西部出现了局部中雨，湛江市廉江市塘莲镇录得单点的大雨25.5毫米；白天我省最高气温普遍上升，其中汕尾惠来录得全省最高气温28.6℃。\n\n具体预报：23日, 韶关、清远、肇庆、云浮晴间多云，粤东多云为主，其余市县多云间阴天，局部有零星小雨。\n\n24日，我省大部多云间晴天气为主，局部有零星小雨，其中受偏东气流影响，雷州半岛有中雨。最低气温：中北部市县18～21℃，南部市县21～23℃。 [...] 25日，粤北的偏北市县、雷州半岛阴天间多云有阵雨，其余大部分市县多云间晴天气为主。最低气温：中北部市县21～23℃，南部市县23～25℃。\n\n26日，粤北的偏北市县、粤西沿海市县阴天有小雨局部中雨，其余市县多云间晴天气为主。最低气温：粤北的偏北市县21～23℃，中北部市县21～23℃，南部市县23～25℃。\n\n海面大风预报：受冷空气影响，23日，我省中东部海面东北风6～7级大风阵风8级，西部海面东北到东风6级。24日开始，我省海面风力减弱。\n\n温馨提醒：1.地质灾害具有滞后性，仍需注意滑坡等地质灾害。2.海面风力大，需注意做好相关防御工作。3

## 上述的举例1也可以改写为:

In [3]:
from langchain import hub
from langchain.schema import prompt_template
import os

from langchain.agents import create_tool_calling_agent, AgentExecutor, create_react_agent

from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
import dotenv
from langchain_community.tools import TavilySearchResults


# 获取Tavily搜索的实例
dotenv.load_dotenv()

search = TavilySearchResults(max_results=3)

# 获取对应的工具的实例
search_tool = Tool(
    func=search.run,
    name='Search',
    description='用于检索互联网上的信息',
)

# 获取大模型
llm = ChatOpenAI(
    model_name=os.getenv("MODEL"),
    api_key=os.getenv("ARK_API_KEY"),
    base_url=os.getenv("BASE_URL"),
    temperature=0
)



prompt_template =  hub.pull("hwchase17/react")

# 获取Agent的实例: create_tool_calling
agent = create_react_agent(
    llm=llm,
    prompt=prompt_template,
    tools=[search_tool],
)

# 获取AgentExecutor的实例
agent_exector =  AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True
)

# 通过AgentExecutor的实例调用invoke(),得到响应
result = agent_exector.invoke({"input":"查询今天汕尾的天气情况"})

print(result)




C:\Users\Admin\.conda\envs\py310\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
我需要查找今天汕尾的天气情况。  
Action: Search  
Action Input: '汕尾 今天的天气'  [{'title': '汕尾 - 中国气象局-天气预报-城市预报', 'url': 'https://weather.cma.cn/web/weather/59501', 'content': '|  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 08:00 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 |\n| 天气 |\n| 气温 | 20.2℃ | 21.8℃ | 23.7℃ | 23.8℃ | 23.7℃ | 21.8℃ | 21℃ | 20.2℃ |\n| 降水 | 无降水 | 2.3mm | 2.3mm | 2.3mm | 2.3mm | 无降水 | 无降水 | 无降水 |\n| 风速 | 7.5m/s | 7.7m/s | 7.5m/s | 7.9m/s | 7.5m/s | 3.3m/s | 2.8m/s | 2.4m/s |\n| 风向 | 东北风 | 东北风 | 东北风 | 东北风 | 东北风 | 东北风 | 东北风 | 东北风 |\n| 气压 | 1010.7hPa | 1010.3hPa | 1009.9hPa | 1010.8hPa | 1011.6hPa | 1011.5hPa | 1011.5hPa | 1012.1hPa | [...] |  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 08:00 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 |\n| 天气 |\n| 气温 | 24.2℃ | 26.2℃ | 29.4℃ | 29.8℃ | 28.4℃ | 27.8℃ | 25.4℃ | 22.6℃ |\n| 降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 2.3mm | 2.3mm | 2.3mm |\n| 风速 | 7.9m/s | 7.9m/s |

## 举例2: 使用ChatPromptTemplate实现

In [5]:
from langchain.schema import prompt_template
import os

from langchain.agents import create_tool_calling_agent, AgentExecutor, create_react_agent

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
import dotenv
from langchain_community.tools import TavilySearchResults


# 获取Tavily搜索的实例
dotenv.load_dotenv()

search = TavilySearchResults(max_results=3)

# 获取对应的工具的实例
search_tool = Tool(
    func=search.run,
    name='Search',
    description='用于检索互联网上的信息',
)

# 获取大模型
llm = ChatOpenAI(
    model_name=os.getenv("MODEL"),
    api_key=os.getenv("ARK_API_KEY"),
    base_url=os.getenv("BASE_URL"),
    temperature=0
)

system_template = template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
"""

# 提供提示词模版(以ChatPromptTemplate为例)
prompt_template =  ChatPromptTemplate.from_messages([
    # ("system","你是一个乐于助人的ai助手,根据用户的提问,必要时调用Search工具,使用互联网检索数据"),
    ("system",system_template),
    ("human","{input}"),
    ("system","{agent_scratchpad}")
])

# 获取Agent的实例: create_tool_calling
agent = create_react_agent(
    llm=llm,
    prompt=prompt_template,
    tools=[search_tool],
)

# 获取AgentExecutor的实例
agent_exector =  AgentExecutor(
    agent=agent,
    tools=[search_tool],
    verbose=True
)

# 通过AgentExecutor的实例调用invoke(),得到响应
result = agent_exector.invoke({"input":"查询今天汕尾的天气情况"})

print(result)






> Entering new AgentExecutor chain...
Question: 查询今天汕尾的天气情况
Thought: 我需要查找今天汕尾的天气信息。
Action: Search
Action Input: "汕尾 今天的天气"[{'title': '汕尾, 廣東省, 中國目前天氣 - AccuWeather', 'url': 'https://www.accuweather.com/zh/cn/shanwei/58218/current-weather/58218', 'content': '返回 \n\n# 汕尾, 廣東省\n\n87°F   \n\n87°\n\n汕尾, 廣東省 天氣\n\n今天 WinterCast 當地{stormName}追蹤 每小時 每天 雷達 MinuteCast® 每月 空氣品質 健康與活動\n\n### 颶風 ### 惡劣天氣 ### 雷達與氣象圖 ### 視訊\n\n今天   每小時   每天   雷達   MinuteCast®   每月   空氣品質   健康與活動\n\n星期六, 10月18日\n\n# 目前天氣\n\nAM10:38\n\n87°F\n\n晴\n\nRealFeel® 101°\n\n酷熱\n\nRealFeel Shade™ 94°\n\n炎熱\n\nRealFeel®\n\n101°\n\nRealFeel Shade™\n\n94°\n\n最大紫外線指數\n\n4.4 (溫和)\n\n風\n\n東北 4英里/小时\n\n風速\n\n11英里/小时\n\n濕度\n\n67%\n\n露點\n\n75° F\n\n氣壓\n\n↓ 29.92英寸汞柱\n\n雲層\n\n10%\n\n能見度\n\n10英里\n\n雲冪\n\n30000英尺\n\n## 白天\n\n 10/18\n\n91°高溫\n\nRealFeel® 103°\n\n酷熱\n\nRealFeel Shade™ 97°', 'score': 0.8616433}, {'title': '汕尾 - 中国气象局-天气预报-城市预报', 'url': 'https://weather.cma.cn/web/weather/59501', 'content': '|  |  |  |  |  |  |  |  |  |\n 